In [ ]:
!cp "/content/drive/MyDrive/Github_stuff/facedetect_pretrained_caffe/deploy.prototxt" "/content"
!cp "/content/drive/MyDrive/Github_stuff/facedetect_pretrained_caffe/res10_300x300_ssd_iter_140000.caffemodel" "/content"
!cp "/content/drive/MyDrive/DataSets/data for face/face.jpeg" "/content"
!cp "/content/drive/MyDrive/DataSets/data for face/face.mp4" "/content"

In [ ]:
# import the necessary packages
import numpy as np
import argparse
import cv2
from imutils.video import VideoStream

import imutils
import time


from google.colab.patches import cv2_imshow
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
image_path="/content/face.jpeg"
proto_path="/content/deploy.prototxt"
model_path="/content/res10_300x300_ssd_iter_140000.caffemodel"
min_confidence=0.70
videoName="/content/face.mp4"
output_video="detections.avi"

print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(proto_path, model_path)


[INFO] loading model...


In [ ]:

image = cv2.imread(image_path)
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
	(300, 300), (104.0, 177.0, 123.0))

[INFO] loading model...


In [ ]:
net.setInput(blob)
detections = net.forward()

In [ ]:

for i in range(0, detections.shape[2]):
    
	confidence = detections[0, 0, i, 2]

    
	if confidence > min_confidence:
		# compute the (x, y)-coordinates of the bounding box for the
		# object
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
 
		# draw the bounding box of the face along with the associated
		# probability
		text = "{:.2f}%".format(confidence * 100)
		y = startY - 10 if startY - 10 > 10 else startY + 10
		image=cv2.rectangle(image, (startX, startY), (endX, endY),
			(0, 0, 255), 2)
		image=cv2.putText(image, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
# show the output image

cv2_imshow(image)

In [ ]:
cap = cv2.VideoCapture(videoName)
ret = True
ret, frame = cap.read()
frame = imutils.resize(frame, width=400)
(h,w)=frame.shape[:2]

fps=cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_video,0,30, (w,h))





In [ ]:

while ret:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels


	frame = imutils.resize(frame, width=400)

	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]

	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
	(300, 300), (104.0, 177.0, 123.0))


	net.setInput(blob)
	detections = net.forward()

	for i in range(0, detections.shape[2]):

		confidence = detections[0, 0, i, 2]

		if confidence < min_confidence:
			continue	

		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")

		text = "{:.2f}%".format(confidence * 100)
		y = startY - 10 if startY - 10 > 10 else startY + 10
		frame=cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
		frame=cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

	out.write(frame)
	
	
	ret, frame = cap.read()
cap.release()
out.release()

print("Video made successfully")

Video made successfully


In [ ]:
!ffmpeg -i "/content/detections.avi" "detections.mp4"

from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/detections.mp4','rb').read()
data_url = "data:/content/detections.mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib